## Imports principais

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm


## Implementação dos modelos para classificação


### Perceptron simples classificação

In [3]:
def perceptron_simples_classification_training(X_test, y_test, X_train, y_train, lr, epochs=100):
    '''
    Treinamento do perceptron simples voltado para classificação.
    Retorna os pesos finais, a acurácia de treino média, a acurácia de teste média, 
    e as listas de acurácia de treino e teste por época.
    '''

    # Instanciando as listas para armazenar a acurácia
    acuracia_treino_list = []
    acuracia_teste_list = []

    # Instanciando pesos
    w = np.zeros(X_train.shape[1])

    for epoch in tqdm(range(epochs)):
        # Fase de treinamento
        acertos_treino = 0

        for x, y_true in zip(X_train, y_train):
            u = np.dot(x, w)
            y_pred = np.sign(u)

            erro = y_true - y_pred

            if y_true == y_pred:
                acertos_treino += 1

            # Atualizando os pesos
            w += lr * erro * x

        # Calculando e armazenando a acurácia de treino
        acuracia_treino = acertos_treino / len(X_train)
        acuracia_treino_list.append(acuracia_treino)

        # Fase de teste
        acertos_teste = 0
        for x, y_true in zip(X_test, y_test):
            u = np.dot(x, w)
            y_pred = np.sign(u)

            if y_true == y_pred:
                acertos_teste += 1

        # Calculando e armazenando a acurácia de teste
        acuracia_teste = acertos_teste / len(X_test)
        acuracia_teste_list.append(acuracia_teste)

    # Calculando a acurácia média de treino e teste
    acuracia_treino_media = np.mean(acuracia_treino_list)
    acuracia_teste_media = np.mean(acuracia_teste_list)

    return w, acuracia_treino_media, acuracia_teste_media, acuracia_treino_list, acuracia_teste_list


### ADALINE classificação

In [ ]:
def adaline_classification_training(X_test, y_test, X_train, y_train, lr, epochs=100):
    '''
    Treinamento do ADALINE voltado para classificação.
    Retorna os pesos finais, a acurácia de treino média, a acurácia de teste média,
    e as listas de acurácia de treino e teste por época.
    '''

    # Instanciando as listas para armazenar a acurácia
    acuracia_treino_list = []
    acuracia_teste_list = []

    # Instanciando pesos
    w = np.zeros(X_train.shape[1])

    for epoch in tqdm(range(epochs)):
        # Fase de treinamento
        acertos_treino = 0

        for x, y_true in zip(X_train, y_train):
            u = np.dot(x, w)
            y_pred = np.sign(u)

            erro = y_true - u

            if y_true == y_pred:
                acertos_treino += 1

            # Atualizando os pesos com a regra de atualização do ADALINE
            w += lr * erro * x

        # Calculando e armazenando a acurácia de treino
        acuracia_treino = acertos_treino / len(X_train)
        acuracia_treino_list.append(acuracia_treino)

        # Fase de teste
        acertos_teste = 0
        for x, y_true in zip(X_test, y_test):
            u = np.dot(x, w)
            y_pred = np.sign(u)

            if y_true == y_pred:
                acertos_teste += 1

        # Calculando e armazenando a acurácia de teste
        acuracia_teste = acertos_teste / len(X_test)
        acuracia_teste_list.append(acuracia_teste)

    # Calculando a acurácia média de treino e teste
    acuracia_treino_media = np.mean(acuracia_treino_list)
    acuracia_teste_media = np.mean(acuracia_teste_list)

    return w, acuracia_treino_media, acuracia_teste_media, acuracia_treino_list, acuracia_teste_list


### Perceptron de múltiplas camadas

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def mlp_classification_training(X_test,
                                y_test,
                                X_train,
                                y_train, 
                                lr, 
                                epochs=100, 
                                hidden_neurons=8):
    '''
    Treinamento de um perceptron de múltiplas camadas voltado para classificação.
    Retorna os pesos finais, a acurácia de treino média, a acurácia de teste média,
    e as listas de acurácia de treino e teste por época.
    '''

    # Inicializando os pesos
    input_neurons = X_train.shape[1]
    output_neurons = len(np.unique(y_train))  # Número de classes

    # Pesos e bias entre camada de entrada e oculta
    W1 = np.random.randn(input_neurons, hidden_neurons)
    b1 = np.zeros((1, hidden_neurons))

    # Pesos e bias entre camada oculta e saída
    W2 = np.random.randn(hidden_neurons, output_neurons)
    b2 = np.zeros((1, output_neurons))

    # Listas para armazenar a acurácia
    acuracia_treino_list = []
    acuracia_teste_list = []

    for epoch in tqdm(range(epochs)):
        # Forward pass
        hidden_input = np.dot(X_train, W1) + b1
        hidden_output = relu(hidden_input)

        final_input = np.dot(hidden_output, W2) + b2
        final_output = sigmoid(final_input)

        # Calculando erro
        y_train_one_hot = np.eye(output_neurons)[y_train]  # Convertendo para one-hot encoding
        error = y_train_one_hot - final_output

        # Backward pass
        d_output = error * sigmoid_derivative(final_output)
        error_hidden_layer = d_output.dot(W2.T)
        d_hidden_layer = error_hidden_layer * relu_derivative(hidden_output)

        # Atualização dos pesos e bias
        W2 += hidden_output.T.dot(d_output) * lr
        b2 += np.sum(d_output, axis=0, keepdims=True) * lr
        W1 += X_train.T.dot(d_hidden_layer) * lr
        b1 += np.sum(d_hidden_layer, axis=0, keepdims=True) * lr

        # Calculando acurácia de treino
        train_predictions = np.argmax(final_output, axis=1)
        acertos_treino = np.sum(train_predictions == y_train)
        acuracia_treino = acertos_treino / len(y_train)
        acuracia_treino_list.append(acuracia_treino)

        # Forward pass para teste
        hidden_input_test = np.dot(X_test, W1) + b1
        hidden_output_test = relu(hidden_input_test)

        final_input_test = np.dot(hidden_output_test, W2) + b2
        final_output_test = sigmoid(final_input_test)

        # Calculando acurácia de teste
        test_predictions = np.argmax(final_output_test, axis=1)
        acertos_teste = np.sum(test_predictions == y_test)
        acuracia_teste = acertos_teste / len(y_test)
        acuracia_teste_list.append(acuracia_teste)

    # Calculando a acurácia média de treino e teste
    acuracia_treino_media = np.mean(acuracia_treino_list)
    acuracia_teste_media = np.mean(acuracia_teste_list)

    return (W1, b1, W2, b2), acuracia_treino_media, acuracia_teste_media, acuracia_treino_list, acuracia_teste_list



## Implementação modelos regressão


### Perceptron Simples para Regressão

In [ ]:
def perceptron_simples_regression_training(X_test, y_test, X_train, y_train, lr, epochs=100):
    '''
    Treinamento do perceptron simples voltado para regressão.
    Retorna os pesos finais, o erro quadrático médio de treino e de teste por época.
    '''

    # Instanciando as listas para armazenar o MSE
    mse_treino_list = []
    mse_teste_list = []

    # Instanciando pesos
    w = np.zeros(X_train.shape[1])

    for epoch in tqdm(range(epochs)):
        # Fase de treinamento
        for x, y_true in zip(X_train, y_train):
            y_pred = np.dot(x, w)
            erro = y_true - y_pred

            # Atualizando os pesos
            w += lr * erro * x

        # Calculando e armazenando o MSE de treino
        y_train_pred = np.dot(X_train, w)
        mse_treino = np.mean((y_train - y_train_pred) ** 2)
        mse_treino_list.append(mse_treino)

        # Calculando e armazenando o MSE de teste
        y_test_pred = np.dot(X_test, w)
        mse_teste = np.mean((y_test - y_test_pred) ** 2)
        mse_teste_list.append(mse_teste)

    mse_treino_media = np.mean(mse_treino_list)
    mse_teste_media = np.mean(mse_teste_list)

    return w,mse_treino_media, mse_teste_media, mse_treino_list, mse_teste_list



### ADALINE para Regressão

In [ ]:
def adaline_regression_training(X_test, y_test, X_train, y_train, lr, epochs=100):
    '''
    Treinamento do ADALINE voltado para regressão.
    Retorna os pesos finais, a média dos erros quadráticos médios de teste e treino,
    o erro quadrático médio de treino e de teste por época.
    '''

    # Instanciando as listas para armazenar o MSE
    mse_treino_list = []
    mse_teste_list = []

    # Instanciando pesos
    w = np.zeros(X_train.shape[1])

    for epoch in tqdm(range(epochs)):
        # Fase de treinamento
        for x, y_true in zip(X_train, y_train):
            y_pred = np.dot(x, w)
            erro = y_true - y_pred

            # Atualizando os pesos com a regra de atualização do ADALINE
            w += lr * erro * x

        # Calculando e armazenando o MSE de treino
        y_train_pred = np.dot(X_train, w)
        mse_treino = np.mean((y_train - y_train_pred) ** 2)
        mse_treino_list.append(mse_treino)

        # Calculando e armazenando o MSE de teste
        y_test_pred = np.dot(X_test, w)
        mse_teste = np.mean((y_test - y_test_pred) ** 2)
        mse_teste_list.append(mse_teste)

    mse_treino_media = np.mean(mse_treino_list)
    mse_teste_media = np.mean(mse_teste_list)

    return w, mse_treino_media, mse_teste_media, mse_treino_list, mse_teste_list



### Multi Layer Perceptron para Regressão

In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def mlp_regression_training(X_test, y_test, X_train, y_train, lr, epochs=100, hidden_neurons=10):
    '''
    Treinamento de um perceptron de múltiplas camadas voltado para regressão.
    Retorna os pesos finais, o erro quadrático médio de treino e de teste por época.
    '''

    # Inicializando os pesos
    input_neurons = X_train.shape[1]
    output_neurons = 1  # Regressão tem uma única saída

    # Pesos e bias entre camada de entrada e oculta
    W1 = np.random.randn(input_neurons, hidden_neurons)
    b1 = np.zeros((1, hidden_neurons))

    # Pesos e bias entre camada oculta e saída
    W2 = np.random.randn(hidden_neurons, output_neurons)
    b2 = np.zeros((1, output_neurons))

    # Listas para armazenar o MSE
    mse_treino_list = []
    mse_teste_list = []

    for epoch in tqdm(range(epochs)):
        # Forward pass
        hidden_input = np.dot(X_train, W1) + b1
        hidden_output = relu(hidden_input)

        final_input = np.dot(hidden_output, W2) + b2
        final_output = final_input  # Regressão linear na camada de saída

        # Calculando erro
        error = y_train - final_output

        # Backward pass
        d_output = error  # Derivada do erro na camada de saída
        error_hidden_layer = d_output.dot(W2.T)
        d_hidden_layer = error_hidden_layer * relu_derivative(hidden_output)

        # Atualização dos pesos e bias
        W2 += hidden_output.T.dot(d_output) * lr
        b2 += np.sum(d_output, axis=0, keepdims=True) * lr
        W1 += X_train.T.dot(d_hidden_layer) * lr
        b1 += np.sum(d_hidden_layer, axis=0, keepdims=True) * lr

        # Calculando o MSE de treino
        mse_treino = np.mean(error ** 2)
        mse_treino_list.append(mse_treino)

        # Forward pass para teste
        hidden_input_test = np.dot(X_test, W1) + b1
        hidden_output_test = relu(hidden_input_test)

        final_input_test = np.dot(hidden_output_test, W2) + b2
        final_output_test = final_input_test

        # Calculando o MSE de teste
        mse_teste = np.mean((y_test - final_output_test) ** 2)
        mse_teste_list.append(mse_teste)

    mse_treino_media = np.mean(mse_treino_list)
    mse_teste_media = np.mean(mse_teste_list)

    return (W1, b1, W2, b2), mse_treino_media, mse_teste_media, mse_treino_list, mse_teste_list

